In [2]:
import numpy as np
import pandas as pd
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.request import Request
from urllib.request import urlopen
import json
import os
import re
import pdb

In [3]:
#Create a function for reading in raw html.
def get_soup(url,header):
    return BeautifulSoup(urlopen(Request(url,headers=header)),'html.parser')
#Setup header for scraping html.
header1 = {'User-Agent':"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.134 Safari/537.36"}

In [11]:
plants.iloc[:1]

,Scientific_Name,Common_Name,Duration,Plant_Type,Sun_Req,Water_Req,State,USDA_ID
0,Abies lasiocarpa,Subalpine Fir,Perennial,Tree,Sun,Moist,AK,ABLA


In [6]:
#Read in plant data from the Lady Bird Conservation website.
plants = pd.read_csv('../Nat_Gard_Build/models/USDA_Lady_Plants.csv', index_col = 0) #Read in native plant data
#Replace any Latin1 encoding with UTF8
for i in range(len(plants)):
    plants['Scientific_Name'][i] = plants['Scientific_Name'][i].replace(u'\xa0', u' ')

In [470]:
#Get state and USDA state-number information to iterate over.
url = 'https://plants.usda.gov/dl_state.html'
html = get_soup(url,header1)
L = []
states = html.find_all("a",href=re.compile("java/stateDownload\?statefips=US\d+"))
for i in range(0,len(states)):
    no = states[i]['href'][31:]
    state = states[i].text
    if(state not in ["District of Columbia","Puerto Rico","Virgin Islands"]):
        L.append([no,state])

L = pd.DataFrame(L,columns = ["No","State"])

In [377]:
#Scrape USDA plant ID and state data.
url = 'https://plants.usda.gov/java/stateDownload?statefips=US' #URL
label = ['Symbol','Scientific_Name','Common_Name','State'] #Labels for the dataset
USDA = [] #List to hold the plant and state data.

for state in range(len(L)):
    t_url = url + L["No"][state]
    html = get_soup(t_url, header1)
    codes = html.get_text()
    codes = codes.split('\n')

    for i in range(1,len(codes)-1):
        row = codes[i].split('",')
        for item in range(len(row)):
            row[item] = row[item].replace('"','')
        if(len(row)==5):
            USDA.append([row[0],row[2],row[3],L["State"][state]])
        else: print(row[0], i, L["State"][state],L["No"][state])

USDA_plant = pd.DataFrame(USDA[1:], columns = label)

In [502]:
#Add the USDA plant symbol to the Lady Bird Johnson conservation dataset.
USDA_ID = []
for plnt in range(len(plants)):
    name = plants['Scientific_Name'][plnt].split(' ')
    rows = USDA_plant
    for nm in range(len(name)):
        rows = rows[rows['Scientific_Name'].str.contains(name[nm])]
    if len(rows) == 0: pdb.set_trace()
    ID = rows['Symbol'].sort_values().unique()[0]
    #ID = rows['Symbol'].value_counts().index[0]
    USDA_ID.append(ID)
    plants['USDA_ID'] = USDA_ID

,Scientific_Name,Common_Name,Duration,Plant_Type,Sun_Req,Water_Req,State,USDA_ID
0,Abies lasiocarpa,Subalpine Fir,Perennial,Tree,Sun,Moist,AK,ABLA
1,Acer circinatum,Oregon Vine Maple,Perennial,Shrub,Shade,Moist,AK,ACCI
2,Acer glabrum,Rocky Mountain Maple,Perennial,Tree,Sun,NaN,AK,ACGL
3,Aconitum columbianum,Columbian Monkshood,Perennial,Herb,Shade,Moist,AK,ACCO4
4,Adiantum pedatum,Northern Maidenhair Fern,Perennial,Fern,Shade,Moist,AK,ADAL
5,Agrostis scabra,Fly-away Grass,Perennial,Grass/Grass-like,Sun,Dry,AK,AGSC5
6,Amelanchier alnifolia,Saskatoon Serviceberry,Perennial,Shrub,Sun,Moist,AK,AMAL2
7,Anaphalis margaritacea,Western Pearly Everlasting,Perennial,Herb,Sun,Dry,AK,ANMA
8,Anemone multifida,Pacific Anemone,Perennial,Herb,Sun,Moist,AK,ANMU
9,Aquilegia formosa,Western Columbine,Perennial,Herb,Sun,Moist,AK,AQFLF
